# IMPORT

In [1]:
# import os
# import yaml
import glob
# import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
# Show all rows from pandas
# pd.set_option('display.max_rows', None)
# import xlwings as xw
%matplotlib inline
# %matplotlib widget
from ipywidgets import interact, interactive, fixed, interact_manual

from dep.config import * # workbook, config, data
from dep.fonctions import (create_main_info_latex, insert_nb_de_cas_dans_latex, create_3D_plot, 
                            create_latex_table, create_latex_cas, fill_cas_in_excel, crop_image)

from dep.group import Group
from dep.cas import Cas

Chargement du fichier Excel TEMPLATE_DDC.xlsx
-----------------------------------------------
Chargement de la configuration des cas de test à partir du fichier Excel
last_row=22, last_col=9, nb_cas=7
{1: {'nom': 'Charges permanentes (G)', 'type_charge': 'individuelle', 'composantes': ['1-G'], 'type_combinaison': 'sum', 'col_ref': 235, 'row_ref': 11, 'groupe': 1, 'type_groupe': 'ponctuel', 'segments_pour_groupe_lineaire': None, 'azimuth': 44.0, 'elevation': 26.0, 'zoom': 1.3, 'coef_RF': 0.4, 'pourcentage_pour_affichage': 0.5, 'top_crop': 500.0, 'bottom_crop': 1440.0, 'left_crop': 500.0, 'right_crop': 1650.0, 'valeur_mini_affichage_RFx_kN': 1.0, 'valeur_mini_affichage_RFy_kN': 1.0, 'valeur_mini_affichage_RFz_kN': 1.0}, 2: {'nom': "Charges d'exploitation (Q)", 'type_charge': 'individuelle', 'composantes': ['2-Q'], 'type_combinaison': 'sum', 'col_ref': 235, 'row_ref': 11, 'groupe': 1, 'type_groupe': 'ponctuel', 'segments_pour_groupe_lineaire': None, 'azimuth': 44.0, 'elevation': 26.0, 'zo

https://matplotlib.org/stable/api/_as_gen/mpl_toolkits.mplot3d.axes3d.Axes3D.stem.html#mpl_toolkits.mplot3d.axes3d.Axes3D.stem

# Liste des fichiers csv dans input_data et nom de groupe dans config

In [2]:
liste_csv = glob.glob('./input_data/*.csv')
print(liste_csv) 
# Méthode par l'analyse des fichiers csv
groupe_fichiers = list()
for f in liste_csv:
    gr_id = int(f.split('\\')[-1].split('_')[0].split(' ')[-1])
    if gr_id not in groupe_fichiers:
        groupe_fichiers.append(gr_id)
print("Liste des groupes dans le répertoire input data:", groupe_fichiers)

# Méthode par la feuille config du fichier excel
groupe = list()
for num_cas, cas in config.items():
    if cas['groupe'] not in groupe and cas['groupe']!=None:
        groupe.append(int(cas['groupe']))
print("Liste des groupes dans le fichier excel:",groupe)

if groupe_fichiers != groupe:
    print("Les groupes ne correspondent pas")

['./input_data\\Groupe 1_Coordonées Points.csv', './input_data\\Groupe 1_DDC combinée.csv', './input_data\\Groupe 1_DDC individuelle.csv', './input_data\\Groupe 2_Coordonées Points.csv', './input_data\\Groupe 2_DDC combinée.csv', './input_data\\Groupe 2_DDC individuelle.csv']
Liste des groupes dans le répertoire input data: [1, 2]
Liste des groupes dans le fichier excel: [1, 2]


# Execution

In [3]:
GROUPES = dict()
# Création du fichier main_infos.tex
create_main_info_latex()
insert_nb_de_cas_dans_latex(groupe=groupe)

for id in groupe:
    print(f"Groupe {id} en début de traitement")
    # Création d'un objet Groupe
    GROUPES[id]= Group(Groupe_id=id)
    # Chargement des coordonnées des points
    GROUPES[id].load_coordonnees()
    # Chargement des charges 
    GROUPES[id].load_ddc_individuelle()
    # Chargement des charges combinées
    GROUPES[id].load_ddc_combinee()
    # Merge des points et des charges individuelles
    GROUPES[id].merge_points_charges()
    # Initialisation de la liste des cas pour ce groupe
    GROUPES[id].dict_cas = dict()
    # Liste des cas uniques de charges individuelles
    print(f'Charges individuelles: {GROUPES[id].df_ddc_individuelle_points.Cas.unique()}')
    # Liste des cas uniques de charges combinées
    print(f'Charges combinées: {GROUPES[id].df_ddc_combinee.Designation.unique()}')
    
    print(f"Création des graphes et des tableaux")
    for numero_cas in config.keys(): # Liste de cas
        if config[numero_cas]['groupe'] == id:
            print(f"Cas {numero_cas} en début de traitement")
            GROUPES[id].dict_cas[numero_cas] = Cas(nom = config[numero_cas]['nom'],
                                                    groupe=id,
                                                    config_cas=config[numero_cas])
            # Création du dataframe de cas
            GROUPES[id].dict_cas[numero_cas].df_cas = GROUPES[id].sort_df(numero_cas=numero_cas)
            # 
            # Calcul du facteur d'échelle et des valeurs min et max des axes ponctuels
            scale_factor, axe_min_max = GROUPES[id].calc_scale_factor(coef=config[numero_cas]['coef_RF'],
                                                                        cas=numero_cas)
            # Création des graphes ponctuel
            GROUPES[id].plot_graphes_reperage_points(numero_cas=numero_cas, axe_min_max=axe_min_max)
            create_3D_plot(df=GROUPES[id].dict_cas[numero_cas].df_cas, 
                        numero_cas=numero_cas, 
                        scale_factor=scale_factor,
                        axe_min_max=axe_min_max)
            # 
            if config[numero_cas]['type_groupe'] == 'lineaire':
                GROUPES[id].dict_cas[numero_cas].create_df_lineaire()
            
            print(f"Cas {numero_cas} terminé")
            print('-----------------------------')
    print(f"Traitement de Groupe {id} terminé")
print('-----------------------------')
print('Cropping des images')
for im in glob.glob('assets/img/*.png'):
    print(im)
    if '3D' in im and 'cropped' not in im:  
        crop_image(img_path=im, top=int(config[numero_cas]['top_crop']), 
                    bottom=int(config[numero_cas]['bottom_crop']), 
                    left=int(config[numero_cas]['left_crop']), 
                    right=int(config[numero_cas]['right_crop']), 
                    show=False)
############################################################################################################################""
print('Création des rapports')
# Selection de la feuille excel
sheet = workbook.sheets['DdC(FRA)']
for id in groupe:
    print(f"Groupe {id} en début d'écriture")    
    # Création des fichiers Latex et excel
    for numero_cas in config.keys(): # Liste de cas
        if config[numero_cas]['groupe'] == id:
            #
            if config[numero_cas]['type_groupe'] == 'ponctuel': 
                if config[numero_cas]['type_charge'] == 'individuelle':
                    df_cas = GROUPES[id].dict_cas[numero_cas].df_cas.loc[:,['Nom point', 'RFx_kN', 'RFy_kN', 'RFz_kN', 'X', 'Y', 'Z']]
                elif config[numero_cas]['type_charge'] == 'combinée':
                    df_cas = GROUPES[id].dict_cas[numero_cas].df_cas.loc[:,['Nom point', 'RFxmin_kN', 'RFxmax_kN', 'RFymin_kN', 'RFymax_kN', 'RFzmin_kN', 'RFzmax_kN', 'X', 'Y', 'Z']]
            elif config[numero_cas]['type_groupe'] == 'lineaire':
                df_cas = GROUPES[id].dict_cas[numero_cas].df_cas_lineique.loc[:,['Segments', 'RFx_kN/ml', 'RFy_kN/ml', 'RFz_kN/ml']]
            # # Création du fichier Latex
            # create_latex_table(df_latex=df_cas, numero_cas=numero_cas)
            # create_latex_cas(numero_cas=numero_cas) 
            # # Remplissage de la feuille excel
            fill_cas_in_excel(groupe=id, numero_cas=numero_cas, sheet=sheet, df_temp=df_cas)


############################################################################################################################""
print('Completed')

# Pour Latex, ne pas oublier de copier les fichiers à la racine puis de compiler by opening the Command Palette (Ctrl + Shift + P) and choosing
# LaTeX Workshop: Build LaTeX Project

Groupe 1 en début de traitement
Charges individuelles: ['1-G' '2-Q' '3-S' '13-W' '15-W3' '17-W5' '19-W7']
Charges combinées: ['ELU-STR' 'ELU-Séisme' 'EX' 'EY']
Création des graphes et des tableaux
Cas 1 en début de traitement
Le Cas choisi est : {'nom': 'Charges permanentes (G)', 'type_charge': 'individuelle', 'composantes': ['1-G'], 'type_combinaison': 'sum', 'col_ref': 235, 'row_ref': 11, 'groupe': 1, 'type_groupe': 'ponctuel', 'segments_pour_groupe_lineaire': None, 'azimuth': 44.0, 'elevation': 26.0, 'zoom': 1.3, 'coef_RF': 0.4, 'pourcentage_pour_affichage': 0.5, 'top_crop': 500.0, 'bottom_crop': 1440.0, 'left_crop': 500.0, 'right_crop': 1650.0, 'valeur_mini_affichage_RFx_kN': 1.0, 'valeur_mini_affichage_RFy_kN': 1.0, 'valeur_mini_affichage_RFz_kN': 1.0}
Lmax 17888.0
min_scale_factor 91.86996045400852
Axe_min -9343.174978172667
Axe_max 44720.0
int(scale_factor_X)=497, int(scale_factor_Y)=417, int(scale_factor_Z)=91
Cas 1 terminé
-----------------------------
Cas 2 en début de traite

In [ ]:
gr = 1
# Liste des attributs de l'objet GROUPES[0]
attributes = [attr for attr in dir(GROUPES[gr]) if not callable(getattr(GROUPES[gr], attr)) and not attr.startswith('__')]
print("Attributes:", attributes)
methods = [method for method in dir(GROUPES[gr]) if callable(getattr(GROUPES[gr], method)) and not method.startswith('__')]
print("Methods:", methods)

In [ ]:
GROUPES[1].dict_cas[10].df_cas.columns

In [ ]:
gr = 2
cas = 9
# Liste des attributs de l'objet GROUPES[0]
attributes = [attr for attr in dir(GROUPES[gr].dict_cas[cas]) if not callable(getattr(GROUPES[gr].dict_cas[cas], attr)) and not attr.startswith('__')]
print("Attributes:", attributes)
methods = [method for method in dir(GROUPES[gr].dict_cas[cas]) if callable(getattr(GROUPES[gr].dict_cas[cas], method)) and not method.startswith('__')]
print("Methods:", methods)

# Pour inserer les graphes dans le document word

In [4]:
from docx import Document
from docx.shared import Inches, Pt
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.table import WD_TABLE_ALIGNMENT


# Open an existing Word document
doc = Document("TEMPLATE_DDC.docx")

# Function to find a specific paragraph
def find_paragraph(doc, search_text):
    for i, paragraph in enumerate(doc.paragraphs):
        if search_text in paragraph.text:
            return i  # Return the index of the paragraph
    return -1  # Not found


# Search for a specific paragraph containing certain text
search_text = "Les descentes de charges sont données selon le repère global, pour chaque cas de charge considéré."
position = find_paragraph(doc, search_text)
position

125

In [5]:
def ajout_image(nom_fichier:str, titre_figure:str, size_inch:int):
    new_paragraph = doc.add_paragraph(titre_figure+"\n", style='Normal')
    image_path = r'C:\Users\nmorand\OneDrive - SIMONIN SAS\_Descente_de_charge\assets\img'+r'\\'+nom_fichier
    run = new_paragraph.add_run()
    run.add_picture(image_path, width=Inches(size_inch))
    new_paragraph.alignment = WD_ALIGN_PARAGRAPH.CENTER

def format_header(cell, font_size=12):
    run = cell.paragraphs[0].runs[0]
    run.bold = True
    run.font.size = Pt(font_size)

def format_cell(cell, font_size=10):
    run = cell.paragraphs[0].runs[0]
    run.font.size = Pt(font_size)

def set_cell_border(cell, **kwargs):
    """
    Set cell borders for top, bottom, left, and right.
    Example usage:
    set_cell_border(cell, top={"sz": 12, "val": "single", "color": "000000", "space": "0"})
    """
    tc = cell._element
    tcPr = tc.get_or_add_tcPr()

    for border_name, border_attrs in kwargs.items():
        border = OxmlElement(f'w:{border_name}')
        for key, value in border_attrs.items():
            border.set(qn(f'w:{key}'), str(value))
        tcPr.append(border)


def ajout_table(df:pd.DataFrame):
    table = doc.add_table(rows=df.shape[0]+1, cols=df.shape[1]) # 'Table Grid'
    # Add the header row (column names)
    for j, col_name in enumerate(df.columns):
        header_cell = table.cell(0, j)
        header_cell.text = col_name
        format_header(header_cell, 12)
        # Apply borders to the header cell
        set_cell_border(
            header_cell,
            top={"sz": 12, "val": "single", "color": "000000", "space": "0"},
            bottom={"sz": 12, "val": "single", "color": "000000", "space": "0"},
            left={"sz": 12, "val": "single", "color": "000000", "space": "0"},
            right={"sz": 12, "val": "single", "color": "000000", "space": "0"}
        )
    # Add the DataFrame content row by row
    for i, row in df.iterrows():
        for j, value in enumerate(row):
            cell = table.cell(i + 1, j)
            cell.text = str(value)
            format_cell(cell, font_size=10)
            # Apply borders to the cell
            set_cell_border(
                cell,
                top={"sz": 6, "val": "single", "color": "000000", "space": "0"},
                bottom={"sz": 6, "val": "single", "color": "000000", "space": "0"},
                left={"sz": 6, "val": "single", "color": "000000", "space": "0"},
                right={"sz": 6, "val": "single", "color": "000000", "space": "0"}
            )
    # Center the table horizontally on the page
    table.alignment = WD_TABLE_ALIGNMENT.CENTER

num_figure = 1
for id in groupe:
    doc.add_heading(f"Repérage Groupe de points {id}\n", level=2)
    ajout_image(nom_fichier='reperage_points_appuis_gr_'+str(id)+'.png', 
                titre_figure="Figure: Repérage des points d'appuis 2D:", 
                size_inch=5)
    #
    ajout_image(nom_fichier='reperage_points_appuis_3D_gr_'+str(id)+'_cropped.png', 
                titre_figure="Figure: Repérage des points d'appuis 3D:", 
                size_inch=5)
    #
    for numero_cas in config.keys():
        if config[numero_cas]['groupe'] == id:
            print(f"Cas {numero_cas} en début d'écriture")
            # TITRE CAS
            doc.add_heading(f"Cas n°{numero_cas} - {config[numero_cas]['nom']}", level=3)
            # Table des données
            if config[numero_cas]['type_groupe'] == 'ponctuel':
                if config[numero_cas]['type_charge'] == 'individuelle':
                    df = GROUPES[id].dict_cas[numero_cas].df_cas.loc[:,['Nom point', 'RFx_kN', 'RFy_kN', 'RFz_kN']]
                elif config[numero_cas]['type_charge'] == 'combinée':
                    df = GROUPES[id].dict_cas[numero_cas].df_cas.loc[:,['Nom point', 'RFxmin_kN', 'RFxmax_kN', 'RFymin_kN', 'RFymax_kN', 'RFzmin_kN', 'RFzmax_kN']]
                ajout_table(df)
            elif config[numero_cas]['type_groupe'] == 'lineaire':
                df = GROUPES[id].dict_cas[numero_cas].df_cas_lineique.loc[:,['Segments', 'RFx_kN/ml', 'RFy_kN/ml', 'RFz_kN/ml']]
                ajout_table(df)
            else:
                print("Type de groupe non reconnu")
            # Erase df
            del df
            # Graphe
            for graphe in ['RFx_kN', 'RFy_kN', 'RFz_kN']:
                ajout_image(nom_fichier='graph3D_charges_cas_'+str(numero_cas)+'_'+graphe.replace('Graphe_','')+'_cropped.png', 
                            titre_figure=f"Figure: Chargement en kN suivant {graphe} \n", 
                            size_inch=5)

    # Save the document
    doc.save("TEST.docx")
    print('Completed')


Cas 1 en début de traitement
Cas 2 en début de traitement
Cas 3 en début de traitement
Cas 4 en début de traitement
Cas 5 en début de traitement
Cas 10 en début de traitement
Cas 9 en début de traitement


In [ ]:
GROUPES[1].dict_cas[10].df_cas.columns

# Pour trouver la bonne orientation des graphes 3D

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import matplotlib.image as mpimg


# bg_img = mpimg.imread('input_data/image_structure.png') 

numero_cas = 1
# Plot 3D plot of the df_ddc_individuelle 
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d', proj_type = 'ortho')

# img = mpimg.imread('input_data/image_structure.png')

# Define the coordinates where the image will be placed (e.g., on the XY plane)
xlim = ax.get_xlim()
ylim = ax.get_ylim()
zval = np.min(ax.get_zlim())  # Lowest z value
# Display the image on the XY plane at z = zval
# ax.imshow(img, extent=[xlim[0], xlim[1], ylim[0], ylim[1]], origin='lower', zorder=-1)

ax.scatter(GROUPES[0].df_coordonnees['X'], GROUPES[0].df_coordonnees['Y'], GROUPES[0].df_coordonnees['Z'], c='r', marker='o')
#
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim(axe_min_max[0],axe_min_max[1])
ax.set_ylim(axe_min_max[0],axe_min_max[1])
ax.set_zlim(axe_min_max[0],axe_min_max[1])
ax.set_axis_off()
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.set_zticklabels([])
ax.set_xticks([])
ax.set_yticks([])
ax.set_zticks([])
# Hide the axes

for i, txt in GROUPES[0].df_coordonnees.iterrows():
    ax.text(txt['X'], txt['Y'],txt['Z'], txt['Nom point'], ha='left', va='bottom')


# Define the update function for sliders
def update(elev, azim, zoom):
    ax.view_init(elev=elev, azim=azim)
    ax.set_box_aspect(None, zoom=zoom)
    fig.canvas.draw_idle()

# Create sliders
elev_slider = widgets.FloatSlider(value=config[numero_cas]['elevation'], min=0, max=90, step=1, description='Elevation:')
azim_slider = widgets.FloatSlider(value=config[numero_cas]['azimuth'], min=0, max=360, step=1, description='Azimuth:')
zoom_slider = widgets.FloatSlider(value=config[numero_cas]['zoom'], min=1, max=10, step=0.1, description='Zoom:')

# Link sliders to update function
widgets.interactive(update, elev=elev_slider, azim=azim_slider, zoom=zoom_slider)

# Display the sliders
display(elev_slider, azim_slider, zoom_slider)


In [ ]:
# Save the workbook (optional: save under a new name to avoid overwriting)
# workbook.save('TEMPLATE_DDC_with_image.xls')

# Close the workbook
# workbook.close()


In [ ]:
# Create a figure and a 2D axis for the image


# Display the image with scaling (adjust extent as needed)
# ax_img.imshow(img, extent=[0.35, 0.7, 0.3, 0.7], aspect='auto')
# ax_img.axis('off')  # Hide the axis for the image

fig = plt.figure()
ax_img = fig.add_axes([0, 0, 1, 1], zorder=-1)  # Full size axis for the image
img = mpimg.imread('input_data/image_structure.png')
# Create a 3D axis over the same figure
ax = fig.add_subplot(111, projection='3d')
ax_img.set_xlim(0, 1)
ax_img.set_ylim(0, 1)
# # Generate some 3D data to plot on top of the background image
# x = np.linspace(-5, 5, 100)
# y = np.linspace(-5, 5, 100)
# x, y = np.meshgrid(x, y)
# z = np.sin(np.sqrt(x**2 + y**2))

# Plot the 3D surface with transparency (alpha)
# ax.plot_surface(x, y, z, cmap='viridis', edgecolor='none', alpha=0.7)
ax.scatter(GROUPES[0].df_coordonnees['X'], GROUPES[0].df_coordonnees['Y'], GROUPES[0].df_coordonnees['Z'], c='r', marker='o')
# Set axis labels (if you want to keep them visible)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
ax.set_zlabel('Z axis')

# Set transparent background for the 3D plot
ax.patch.set_alpha(0)  # Make the 3D axis background transparent

# Set the pane colors (the background planes) to be fully transparent
ax.xaxis.pane.set_edgecolor((1, 1, 1, 0))  # X axis background
ax.yaxis.pane.set_edgecolor((1, 1, 1, 0))  # Y axis background
ax.zaxis.pane.set_edgecolor((1, 1, 1, 0))  # Z axis background

ax.xaxis.pane.fill = False  # Turn off the X plane's background fill
ax.yaxis.pane.fill = False  # Turn off the Y plane's background fill
ax.zaxis.pane.fill = False  # Turn off the Z plane's background fill

# Remove the grid and spines to make it cleaner
ax.grid(False)
ax.set_frame_on(False)

# Adjust axis limits to position the 3D plot properly on the background
ax.set_xlim(axe_min_max[0],axe_min_max[1])
ax.set_ylim(axe_min_max[0],axe_min_max[1])
ax.set_zlim(axe_min_max[0],axe_min_max[1])



def update(x_img_min, x_img_max, y_img_min, y_img_max):

    ax_img.imshow(img, extent=[x_img_min, x_img_max, y_img_min, y_img_max], aspect='auto')
    fig.canvas.draw_idle()

# Create sliders
x_img_min = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.05, description='x_img_min:')
x_img_max = widgets.FloatSlider(value=0.75, min=0, max=1, step=0.05, description='x_img_max:')
y_img_min = widgets.FloatSlider(value=0.25, min=0, max=1, step=0.05, description='y_img_min:')
y_img_max = widgets.FloatSlider(value=0.6, min=0, max=1, step=0.05, description='y_img_max:')

# Link sliders to update function
widgets.interactive(update, x_img_min=x_img_min, x_img_max=x_img_max, y_img_min=y_img_min, y_img_max=y_img_max)

# Display the sliders
display(x_img_min, x_img_max, y_img_min, y_img_max)




# Rework image

In [ ]:
imgs =glob.glob('assets/img/*.png')
for i, f, in enumerate(imgs):
    print(i, f)

In [ ]:
crop_image(img_path=imgs[42], top=500, bottom=1440, left=500, right=1650, show=True)

In [ ]:
for im in glob.glob('assets/img/*.png'):
    print(im)
    if ('3D' in im) and 'cropped' not in im:    
        crop_image(img_path=im, top=500, bottom=1440, left=500, right=1650, show=True)